## How to extract CMIP5 data for the North-West European Shelf

This notebook shows how to extract any CMIP5 data for the north-west European Shelf. The example of monthly SST from the HadGEM2-ES model will be used, but it can be easily adapted to deal with any model and any variable. 


You will likely need to install a few packages to get this working. So just run the following

    conda install nctoolkit -y
    pip install git+https://github.com/r4ecology/cmipsearch.git
    pip install pyesg

In [ ]:
import cmipsearch
import random
import nctoolkit as nc
import pandas as pd
import xarray as xr
import numpy as np
nc.options(lazy = True)
import os

## Log on to ESGF

Some of the files require you to be logged on to access them. If you do not have an id, get one here: https://esgf-node.llnl.gov/search/cmip5/.

The data available in this tutorial does not require that you log in, so ignore this if you do not want to register.

In [ ]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()
lm.logon(hostname='esgf-node.llnl.gov', interactive=True, bootstrap=True)
lm.is_logged_on()


CMIP5 data is available from multiple nodes, so if you want to get all available data, you need to search all nodes. The small package cmipsearch available on the PML modelling group's GitHub page will do just this. For this tutorial, we will extract HadGEM2-ES SST data for the historical time period, which is before 2006, and RCP 8.5 which is 2006 onwards. (Or not quite, because the Hadley Centre's historical scenario mysteriously ends in November 2005....)

In [ ]:
ensemble = []
for rcp in ["historical",  "rcp85"]:
    ensemble.append(cmipsearch.cmip5_search(frequency="mon", var = "tos", experiment=[rcp], models = "HadGEM2-ES"))
ensemble = pd.concat(ensemble)

We'll then restrict the files to the years from 1985 to 2100:

In [ ]:
ensemble = ensemble.query("start < 2101").query("end > 1985").reset_index(drop = True)

The files are available via OPeNDAP. This means that you do not need to download global data. Instead you can select a sub-region that will include the north-west European shelf. We can do this using NCO. Note: CDO is not well suited to this as it can only extract the entire planet.

But first, we need a way to generate the NCO call. We just need to derive the call once for a model we are dealing with. The function below will do this for the NW shelf. In essence, it will extract the global data for the first time step, and then use that to figure out the NCO call that will give you the NW shelf. It's a bit hacked together, but it does the job

In [ ]:

# Download the global data for the 1st time step
def generate_call(ff, lon_range, lat_range):
    if "MIROC4h" in ff:
        return "ncks -d rlon,280.,400. -d rlat,40.,80."
    data = nc.open_thredds(ff)
    data.select(timesteps=0)
    data.run()
    # Work out the name of longitude and latitude
    if len([x for x in list(data.to_xarray().dims) if "lon" in x]) == 1:
        lon_name = [x for x in list(data.to_xarray().dims) if "lon" in x ][0]
    else:
        lon_name = None
    if len([x for x in list(data.to_xarray().dims) if "lat" in x]) == 1:
        lat_name = [x for x in list(data.to_xarray().dims) if "lat" in x ][0]
    else:
        lat_name = None
    
    ds= data.to_xarray() 
    lon_coord = [x for x in ds.coords if "lon" == x ][0]
    if (lon_name is not None) and (len(ds[lon_coord].values.shape) == 1):
        if len(np.shape(ds[lon_name].values)) == 1:
            dimensions = 1
        else:
            dimensions = 2
            
        units = ds[lon_name].attrs["units"]
        
        # 1D case
        
        if dimensions == 1:
            # if units are in degrees east, we need to use wrapping coordinates
            if units == "degrees_east":
                if lon_range[0] < 0:
                    lon_min = 360 + lon_range[0]
                else:
                    lon_min = lon_range[0]
                    
                if lon_range[1] < 0:
                    lon_max = 360 + lon_range[1]
                else:
                    lon_max = lon_range[1]
                    
                lat_min = lat_range[0]
                lat_max = lat_range[1]
            else:
                lon_min = lon_range[0]
                lon_max = lon_range[1]
                lat_min = lat_range[0]
                lat_max = lat_range[1]
            
            if lon_name == "rlon":
                if min(ds[lon_name].values) > 0:
                    lon_min = 310 
                    lon_max = 360
                    lat_min = 30
                    lat_max = 80
                else:
                    lon_min = lon_range[0]
                    lon_max = lon_range[1]
            
            lon_min = float(lon_min)
            lon_max = float(lon_max)
            lat_min = float(lat_min)
            lat_max = float(lat_max)
                
                
            # If it's rotated, we need this:
            return f"ncks -d {lon_name},{lon_min},{lon_max} -d {lat_name},{lat_min},{lat_max}"
        
    ds= data.to_xarray() 
    units = ds["lon"].attrs["units"]
    if "east" in units:
        
        ds = data.to_xarray()
        lon_coord = [x for x in data.to_xarray().coords if "lon" in x ][0]
        if len(ds[lon_coord].values.shape) > 1:
            lon_coord
            lat_coord = [x for x in data.to_xarray().coords if "lat" in x ][0]
            list(ds[lat_coord].dims)
            lon_min = lon_range 
            lat_min = lat_range 
            df1 = (
            ds
                .coords[lon_coord]
                .to_dataframe()
                .loc[:,[lon_coord]]
                .iloc[:,1:]
                .reset_index()
            #     .query("lon  > 13")
                .query(f"{lon_coord} < {lon_min[1]} | {lon_coord} > {lon_min[0] + 360}")
                .drop(columns = [lon_coord])
                .reset_index(drop = True)
            )
            df2 = (
            ds
                .coords[lat_coord]
                .to_dataframe()
                .loc[:,[lat_coord]]
                .iloc[:,1:]
                .reset_index()
            #     .query("lon  > 13")
                .query(f"{lat_coord} > {lat_min[0]}")
                .query(f"{lat_coord} < {lat_min[1]}")
                .drop(columns = [lat_coord])
                .reset_index(drop = True)
            )
            merged = df1.merge(df2) 
            
            command = f"ncks "
            
            for cc in merged.columns:
                cc_min = min(merged.loc[:,[cc]].values)[0]
                cc_max = max(merged.loc[:,[cc]].values)[0]
                command = f"{command} -d {cc},{cc_min},{cc_max}"
            command
    return command
        

Now let's look at the files:

In [ ]:
ensemble

We have a total of 10 files for HadGEM2-ES. But these cover multiple variants, i.e. starting conditions. Let's work with variant r1i1p1.

In [ ]:
ensemble = ensemble.query("variant == 'r1i1p1'").reset_index(drop = True)
ensemble

Right now cmipsearch only provides urls for downloads, not the OpenDAP urls. But this can be fixed easily:

In [ ]:
ensemble["url"] = [ff.replace("fileServer","dodsC") for ff in ensemble.url]

This gives us 3 files. One covers the historical period. The others cover the RCP period. 

We now need to generate the call to extract cropped versions of the data:

In [ ]:
nco_call = generate_call(ensemble.url[0], lon_range = [-25, 25], lat_range = [35, 75])

The call is as follows. NCO will crop longitudes between 335 and 25. Essentially 335 is 25 degrees west. Longitude is "degrees east" in HadGEM2-ES, and NCO is not smart enough to figure this out itself, so you need to use wrapped coordinates. The latitudes in the call are 35 and 75 as you would expect.

In [ ]:
nco_call

We can now use nctoolkit to put all of these files into a single dataset.

In [ ]:
sst = nc.open_thredds(ensemble["url"])

We need to call NCO to extract the data:

In [ ]:
sst.nco_command(nco_call)

In theory, this can be done in parallel. I need to work out how sensible that is, as parallel processing with OpenDAP servers can be volatile.

We now want to merge the files. Note that we first need to crop it using CDO. This will convert the file into a more sensible format, to tidy up NCO's output.

In [ ]:
sst.crop(lon_range = [-25, 25], lat_range = [35, 75])
sst.merge_time()
sst.plot()

We now have a dataset with projected SST changes under RCP 8.5. And we can then do things like plot the spatial averaged annual SST:

In [ ]:
sst.spatial_mean()
sst.annual_mean()
sst.plot()